# Training GNNs to detect patient 0

## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import ndlib
import numpy as np

import plotly.graph_objects as go

## Training Data

In [3]:
# Generate 50 graphs, each with 50 nodes
NUM_NODES = 50
NUM_EDGES = 100
NUM_ITERATIONS = 10
NUM_TRAINING = 1000
BETA = 0.15
GAMMA = 0

graphs = np.zeros((NUM_TRAINING, NUM_NODES, NUM_NODES))
nodes_statuses = []
initial_infected = []
for i in range(NUM_TRAINING):
    graph = nx.gnm_random_graph(NUM_NODES, NUM_EDGES)
    p0= np.random.randint(NUM_NODES)

    config = mc.Configuration()
    config.add_model_initial_configuration("Infected", [p0])
    config.add_model_parameter("beta", BETA)
    config.add_model_parameter("gamma", GAMMA)

    model = ep.SIRModel(graph)
    model.set_initial_status(config)

    iterations = model.iteration_bunch(NUM_ITERATIONS)
    statuses = [iteration['status'] for iteration in iterations]
    union_of_statuses = {k:0 for k in range(NUM_NODES)}
    for status in statuses:
        union_of_statuses.update(status)

    adj_matr = np.array(nx.adjacency_matrix(graph).todense())
    graphs[i,:,:] = adj_matr  # Convert sparse matrix to dense matrix
    nodes_statuses.append([v for _,v in sorted(union_of_statuses.items(), key=lambda item: item[0])])
    initial_infected.append([1 if i==p0 else 0 for i in range(NUM_NODES)])




/var/folders/hp/_5r1484j457cv8ysknrcyhvr0000gn/T/ipykernel_91773/4206117842.py:30: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matr = np.array(nx.adjacency_matrix(graph).todense())


## Graph Visualization

In [47]:
G = graph
edge_x = []
edge_y = []
positions = nx.spring_layout(G)
for edge in G.edges():
    x0, y0 = positions[edge[0]]
    x1, y1 = positions[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

if True:
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = positions[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=[],
            # size = 10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line=dict(
                color='Black',
                width=2
            )))

    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append('# of connections: '+str(len(adjacencies[1])))

    avg_degree = {
        0: np.average([adj for i, adj in enumerate(node_adjacencies) if not nodes_statuses[-1][i]]),
        1: np.average([adj for i, adj in enumerate(node_adjacencies) if nodes_statuses[-1][i]]),
        2: np.average([adj for i, adj in enumerate(node_adjacencies)])
    }

    node_trace.marker.color = node_adjacencies
    # node_trace.marker.color = nodes_statuses[-1]
    node_trace.marker.size = [20 if nodes_statuses[-1][i] else 10 for i in range(NUM_NODES)]
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title=f'Network Snapshot of Graph with {NUM_NODES} Nodes and {NUM_EDGES} Edges',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text=f'Average degree of nodes is {avg_degree[2]} and average infected degree is  {avg_degree[1]:2.2f}',
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ), ],
                        
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()

In [44]:
# Define the GCN layer
class GraphConvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolutionLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))

        # Initialize learnable parameters
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj_matrix):
        # Perform graph convolution
        support = torch.mm(x, self.weight)
        output = torch.mm(adj_matrix, support)  # Propagate information through the graph
        output = output + self.bias
        return output


In [57]:

class GraphConvolutionalNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(GraphConvolutionalNetwork, self).__init__()
        self.gcn1 = nn.Sequential(
            GraphConvolutionLayer(input_dim, hidden_dim1),
            nn.ReLU()
        )
        self.gcn2 = nn.Sequential(
            GraphConvolutionLayer(hidden_dim1, hidden_dim2),
            nn.ReLU()
        )
        self.gcn3 = nn.Sequential(
            GraphConvolutionLayer(hidden_dim2, output_dim),
            nn.Softmax(dim=1)
        )

    def forward(self, x, adj_matrix):
        x = self.gcn1(x, adj_matrix)
        x = self.gcn2(x, adj_matrix)
        x = self.gcn3(x, adj_matrix)
        return x

# Define a simple training loop
def train_model(model, features, adj_matrix, labels, num_epochs, learning_rate):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(features, adj_matrix)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    return loss.item()
    # print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

In [58]:

num_nodes = NUM_NODES
num_features = 1
num_classes = NUM_NODES
model = GraphConvolutionalNetwork(num_features, 16,16, num_classes)
print(model)
overall_training_loss = []
overall_testing_loss = []
criterion = nn.CrossEntropyLoss()
# for epoch in range(100):
#     epoch_loss = 0
#     model.train()
#     for i in range(int(NUM_TRAINING*0.7)):
#         features = torch.tensor(nodes_statuses[i]).reshape((NUM_NODES,1)).float()
#         adjacency_matrix = torch.tensor(graphs[i]).float()
#         labels = torch.tensor(initial_infected[i])
#         epoch_loss += train_model(model, features, adjacency_matrix, labels, num_epochs=1, learning_rate=0.01)
#     overall_training_loss.append(epoch_loss)
#     print(f"Avergae loss of epoch {epoch}: {epoch_loss/(NUM_TRAINING*0.7)}")

#     test_loss = 0
#     correct = 0
#     total = 0
#     model.eval()
#     for i in range(int(NUM_TRAINING*0.7), NUM_TRAINING):
#         features = torch.tensor(nodes_statuses[i]).reshape((NUM_NODES,1)).float()
#         print(features)
#         adjacency_matrix = torch.tensor(graphs[i]).float()
#         print(adjacency_matrix)
#         labels = torch.tensor(initial_infected[i])
#         print(labels)
#         with torch.no_grad():
#             output = model.forward(features, adjacency_matrix)
#             print(output)
#             # test_loss += criterion(output, labels).item()
#             # _, predicted = torch.max(output.data, 1)
#             # total += labels.size(0)
#             # correct += (predicted == labels).sum().item
#         break
#     overall_testing_loss.append(test_loss)
#     print(f"Avergae error of epoch {epoch}: {test_loss/(NUM_TRAINING * 0.3)}")
#     print('Accuracy of the network on the test data for epoch {}: {} %'.format(epoch, 100 * correct / total))


GraphConvolutionalNetwork(
  (gcn1): Sequential(
    (0): GraphConvolutionLayer()
    (1): ReLU()
  )
  (gcn2): Sequential(
    (0): GraphConvolutionLayer()
    (1): ReLU()
  )
  (gcn3): Sequential(
    (0): GraphConvolutionLayer()
    (1): Softmax(dim=1)
  )
)


In [65]:
features = torch.tensor(nodes_statuses[1]).reshape((NUM_NODES,1)).float()
adjacency_matrix = torch.tensor(graphs[1]).float()
labels = torch.tensor(initial_infected[1])

model.eval()
with torch.no_grad():
    output = model.forward(features, adjacency_matrix)
    print(output)

TypeError: Sequential.forward() takes 2 positional arguments but 3 were given

In [27]:
import plotly.graph_objs as go
from plotly.offline import iplot

# Create traces
trace0 = go.Scatter(
    x = list(range(100)),
    y = overall_training_loss,
    mode = 'lines',
    name = 'Training Loss'
)

trace1 = go.Scatter(
    x = list(range(100)),
    y = overall_testing_loss,
    mode = 'lines',
    name = 'Testing Loss'
)

data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Training and Testing Loss over Epochs',
              xaxis = dict(title = 'Epoch'),
              yaxis = dict(title = 'Loss'),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='line-mode')
